In [62]:
# import necessary libraries

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib as plt
git_url = 'https://github.com/knnthfrmn/dsc25_group.git'

In [63]:
# load all data

unemployment_data = pd.read_csv('data/global_unemployment_data.csv')
happiness_2015 = pd.read_csv('data/happiness data/2015.csv')
happiness_2016 = pd.read_csv('data/happiness data/2016.csv')
happiness_2017 = pd.read_csv('data/happiness data/2017.csv')
happiness_2018 = pd.read_csv('data/happiness data/2018.csv')
happiness_2019 = pd.read_csv('data/happiness data/2019.csv')

# add year column to happiness dataframes

happiness_2015['Year'] = 2015
happiness_2016['Year'] = 2016
happiness_2017['Year'] = 2017
happiness_2018['Year'] = 2018
happiness_2019['Year'] = 2019


In [64]:
happiness_2015 = happiness_2015.rename(columns={'Economy (GDP per Capita)': 'GDP per cap', 
                                                'Health (Life Expectancy)': 'Life Expectancy', 
                                                'Trust (Government Corruption)': 'Corruption'})

In [65]:
happiness_2015 = happiness_2015.drop(columns={'Region', 'Standard Error', 'Dystopia Residual'})

In [66]:
happiness_2016 = happiness_2016.rename(columns={'Economy (GDP per Capita)': 'GDP per cap', 
                                                'Health (Life Expectancy)': 'Life Expectancy', 
                                                'Trust (Government Corruption)': 'Corruption'})

In [67]:
happiness_2016 = happiness_2016.drop(columns={'Region', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Dystopia Residual'})

In [68]:
happiness_2017 = happiness_2017.rename(columns={'Happiness.Rank': 'Happiness Rank', 
                                                'Happiness.Score': 'Happiness Score', 
                                                'Economy..GDP.per.Capita.': 'GDP per cap',
                                                'Health..Life.Expectancy.': 'Life Expectancy',
                                                'Trust..Government.Corruption.': 'Corruption'})

In [69]:
happiness_2017 = happiness_2017.drop(columns={'Whisker.high', 'Whisker.low', 'Dystopia.Residual'})

In [70]:
happiness_2018 = happiness_2018.rename(columns={'GDP per capita': 'GDP per cap',
                                                'Country or region': 'Country',
                                                'Score': 'Happiness Score',
                                                'Overall rank': 'Happiness Rank',
                                                'Social support': 'Family',
                                                'Healthy life expectancy': 'Life Expectancy',
                                                'Perceptions of corruption': 'Corruption',
                                                'Freedom to make life choices': 'Freedom'})

In [71]:
happiness_2019 = happiness_2019.rename(columns={'GDP per capita': 'GDP per cap',
                                                'Country or region': 'Country',
                                                'Score': 'Happiness Score',
                                                'Overall rank': 'Happiness Rank',
                                                'Social support': 'Family',
                                                'Healthy life expectancy': 'Life Expectancy',
                                                'Perceptions of corruption': 'Corruption',
                                                'Freedom to make life choices': 'Freedom'})

In [72]:
# merge all happiness data by year into one big dataframe
happiness_data = pd.concat([happiness_2015, happiness_2016, happiness_2017, happiness_2018, happiness_2019])

In [73]:
happiness_data = happiness_data.reset_index()
happiness_data.head()

,index,Country,Happiness Rank,Happiness Score,GDP per cap,Family,Life Expectancy,Freedom,Corruption,Generosity,Year
0,0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2015
1,1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2015
2,2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2015
3,3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2015
4,4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2015


In [74]:
happiness_data = happiness_data.drop(columns={'index'})
happiness_data.head()

,Country,Happiness Rank,Happiness Score,GDP per cap,Family,Life Expectancy,Freedom,Corruption,Generosity,Year
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2015
1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2015
2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2015
3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2015
4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2015
